Import numpy to get things started

In [2]:
import numpy as np

Then define the dimensions of the robot

In [3]:
WHEEL_RADIUS = 0.030
TRACK_WIDTH  = 0.195 
TRACK_LENGTH = 0.150

# print(f"Calculating wheel speeds for {wz}, {vx}, {vy}")
l = TRACK_LENGTH / 2
w = TRACK_WIDTH / 2
r = WHEEL_RADIUS


Now lets define a couple of kinematic models to play with. `kinematic_model_0` is the one implemented as per The Construct's tutorial, it differs from the column order in the [paper](https://cdn.intechopen.com/pdfs/465/InTechOmnidirectional_mobile_robot_design_and_implementation.pdf) though. Let's play around with the ordering and see if the values come out the same.

In [21]:
# The kinematic model assumes that the motors are laid out as follows:
#  [3]    [0]
#     ---> x
#  [2]    [1]
#
# NE-Five is wired up as follows:
#  [2]    [1]
#     ---> x
#  [3]    [0]

kinematic_model_0 = np.array([[-l-w, 1, -1],
                              [ l+w, 1,  1],
                              [ l+w, 1, -1],
                              [-l-w, 1,  1]]) / r


kinematic_model_1 = np.array([[-l-w, -1, 1],
                              [ l+w,  1, 1],
                              [ l+w, -1, 1],
                              [-l-w,  1, 1]]) / r

# From page 13: https://cdn.intechopen.com/pdfs/465/InTechOmnidirectional_mobile_robot_design_and_implementation.pdf
def steering(kinematic_model, vx, vy, wz):
    input_array = np.array([wz,vx,vy])    
    input_array.shape = (3, 1)

    u = np.dot(kinematic_model, input_array)
    print(f"u: {u}")
    
    return [u[0][0],u[1][0],u[2][0],u[3][0]]

In [5]:
wheel_speeds_0 = steering(kinematic_model_0, 0.1, 0.1, 0.1)
wheel_speeds_1 = steering(kinematic_model_1, 0.1, 0.1, 0.1)

print(f"Wheel speeds for kinematic model 0: {wheel_speeds_0}")
print(f"Wheel speeds for kinematic model 1: {wheel_speeds_1}")

Wheel speeds for kinematic model 0: [array([7.24166667]), array([-0.575]), array([6.09166667]), array([0.575])]
Wheel speeds for kinematic model 1: [array([7.24166667]), array([-0.575]), array([6.09166667]), array([0.575])]


So, the column order doesn't seem to matter! How about the row order as each row represents a different motor?

In [6]:
kinematic_model_2 = np.array([[ l+w, 1,  1],
                              [-l-w, 1, -1],
                              [ l+w, 1, -1],
                              [-l-w, 1,  1]]) / r

wheel_speeds_1 = steering(kinematic_model_2, 0.1, 0.1, 0.1)
print(f"Wheel speeds for kinematic model 2: {wheel_speeds_1}")

Wheel speeds for kinematic model 2: [array([-0.575]), array([7.24166667]), array([6.09166667]), array([0.575])]


Perfect! So, for the motor layout on NE-Five the array would be as follows:

In [28]:


front_left  = [-l-w, 1, -1]
front_right = [ l+w, 1,  1]
rear_left   = [-l-w, 1,  1]
rear_right  = [ l+w, 1, -1]

top    = [ 1,  1,  1,  1]
middle = [-1,  1,  1, -1]
bottom = [1/(-l-w), 1/(l+w), 1/(l+w), 1/(-l-w)]

forward_kinematic_model = np.array([front_left, 
                                    front_right,
                                    rear_left,
                                    rear_right])

fkm_actual = np.multiply(forward_kinematic_model, 1/r)

ikm = np.array([top, middle, bottom])
ikm_actual = np.multiply(ikm, r/4)

print(f"inverse kinematic model: {ikm_actual}")

x, y, z = 1, 0, 0

wheel_speeds_ne5 = steering(fkm_actual, x, y, z)
print(f"input velocities: {x}, {y}, {z}")
print(f"wheel speeds: {wheel_speeds_ne5}")

input_array = np.array([[wheel_speeds_ne5[0], wheel_speeds_ne5[1], wheel_speeds_ne5[2], wheel_speeds_ne5[3]]])
print(f"input_array: {input_array}, shape: {input_array.shape}")
input_array.shape = (4, 1)
velocities = np.multiply(ikm_actual, input_array)
print(f"velocities: {velocities}")

inverse kinematic model: [[ 0.0075      0.0075      0.0075      0.0075    ]
 [-0.0075      0.0075      0.0075     -0.0075    ]
 [-0.04347826  0.04347826  0.04347826 -0.04347826]]
u: [[33.33333333]
 [33.33333333]
 [33.33333333]
 [33.33333333]]
input velocities: 1, 0, 0
wheel speeds: [33.333333333333336, 33.333333333333336, 33.333333333333336, 33.333333333333336]
input_array: [[33.33333333 33.33333333 33.33333333 33.33333333]], shape: (1, 4)


ValueError: operands could not be broadcast together with shapes (3,4) (4,1) 